# Assignment 5 - Population Analyis

### Authored by: Stephen Kerr

This assingment is broken down into 3 parts.
1. Differences between the sexes by age in Ireland,
2. Grouped population within 5 years of a given varaible age, then look at the differences between them,
3. Regional Differences between the sexes to identify which region has be biggest difference for a given age group,

This Notebook will be divided into 3 Parts to address each assessment part seperately. 

The Population data analysis was gathered from the Irish CSO from the 2022 Census.  
See their website [here](https://www.cso.ie/en/index.html).

## Environemnt Setup

In [1]:
# Imports 

import pandas as pd 

## Load the Data

In [17]:
url = "https://ws.cso.ie/public/api.restful/PxStat.Data.Cube_API.ReadDataset/FY006A/CSV/1.0/en"
df = pd.read_csv(url)
df.tail()
df.to_csv("raw_population_data.csv")

## Part 1

Brief: 
> Write a jupyter notebook that analyses the differences between the sexes by age in Ireland.  
> Weighted mean age (by sex)  
> The difference between the sexes by age  
>This part does not need to look at the regions.  

In [3]:
# Checking the headers of the dataframe

headers = df.columns.to_list()
headers

['STATISTIC',
 'Statistic Label',
 'TLIST(A1)',
 'CensusYear',
 'C02199V02655',
 'Sex',
 'C02076V03371',
 'Single Year of Age',
 'C03789V04537',
 'Administrative Counties',
 'UNIT',
 'VALUE']

In [4]:
# Dropping unnecessary columns from the dataframe

drop_headers_list = ['STATISTIC',
 'Statistic Label',
 'TLIST(A1)',
 'CensusYear',
 'C02199V02655',
 'C02076V03371',
 'C03789V04537',
 'UNIT',
]
df.drop(columns=drop_headers_list, inplace=True)
df.head()

,Sex,Single Year of Age,Administrative Counties,VALUE
0,Both sexes,All ages,Ireland,5149139
1,Both sexes,All ages,Carlow County Council,61968
2,Both sexes,All ages,Dublin City Council,592713
3,Both sexes,All ages,Dún Laoghaire Rathdown County Council,233860
4,Both sexes,All ages,Fingal County Council,330506


In [ ]:
# Cleaning up the 'Single Age' column

df = df[df["Single Year of Age"] != "All ages"] # Remove the 'All ages' rows
df['Single Year of Age'] = df['Single Year of Age'].str.replace('Under 1 year', '0') # Replace 'Under 1 year' with '0'
df['Single Year of Age'] = df['Single Year of Age'].str.replace(r'\D', '', regex=True) # Remove any non-digit characters using regex
# df['Single Year of Age'].unique() # Check unique values after cleaning

In [14]:
# Converting 'Single Year of Age' to integer type

# df.info() # checking the data types and non-null counts
df['Single Year of Age'] = df['Single Year of Age'].astype('Int64') # Convert 'Single Year of Age' to integer
df.info() # checking the data types and non-null counts after conversion

<class 'pandas.core.frame.DataFrame'>
Index: 9696 entries, 32 to 9791
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Sex                      9696 non-null   object
 1   Single Year of Age       9696 non-null   Int64 
 2   Administrative Counties  9696 non-null   object
 3   VALUE                    9696 non-null   int64 
dtypes: Int64(1), int64(1), object(2)
memory usage: 388.2+ KB


In [ ]:
# Creating a Pivot Table to analyze population by Age, Sex and Administrative Counties

# added .astype(int) to convert float to int as I was getting float values 
df_analysis = pd.pivot_table( df, 'VALUE', "Single Year of Age", "Administrative Counties").astype(int) 
# df_analysis.info()
df_analysis.to_csv("population_by_age_and_county.csv")

<class 'pandas.core.frame.DataFrame'>
Index: 102 entries, 1 year to Under 1 year
Data columns (total 32 columns):
 #   Column                                 Non-Null Count  Dtype
---  ------                                 --------------  -----
 0   Carlow County Council                  102 non-null    int64
 1   Cavan County Council                   102 non-null    int64
 2   Clare County Council                   102 non-null    int64
 3   Cork City Council                      102 non-null    int64
 4   Cork County Council                    102 non-null    int64
 5   Donegal County Council                 102 non-null    int64
 6   Dublin City Council                    102 non-null    int64
 7   Dún Laoghaire Rathdown County Council  102 non-null    int64
 8   Fingal County Council                  102 non-null    int64
 9   Galway City Council                    102 non-null    int64
 10  Galway County Council                  102 non-null    int64
 11  Ireland                